In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train = pd.read_csv(r'train.csv', encoding='utf_8_sig')
train.fillna('', inplace=True)
test = pd.read_csv(r'test.csv', encoding='utf_8_sig')
test.fillna('', inplace=True)

In [3]:
train

,label,text,diff
0,positive,Check types to avoid invalid reads/writes.,diff --git a/src/file.c b/src/file.c\nindex 4d...
1,positive,[CVE-2012-2098] Integrate fallback sort into t...,diff --git a/src/main/java/org/apache/commons/...
2,negative,Merge branch '1.8.x',diff --git a/src/ClientCommand.cpp b/src/Clien...
3,negative,[JBPM-9474] Produce events to Kafka by Signal ...,diff --git a/jbpm-bpmn2/src/main/java/org/jbpm...
4,positive,Resolving Issue #918\n\nMinor XSS and create g...,diff --git a/docs/CHANGELOG b/docs/CHANGELOG\n...
5,positive,CVE-2017-12895/ICMP: Check the availability of...,diff --git a/print-icmp.c b/print-icmp.c\ninde...
6,negative,9.0.4,diff --git a/package.json b/package.json\ninde...
7,negative,"Needs to allow this syscall as well, caught on...",diff --git a/linux_priv.c b/linux_priv.c\ninde...
8,positive,SPOOLSS: Try to avoid an infinite loop.\n\nUse...,diff --git a/epan/dissectors/packet-dcerpc-spo...
9,positive,Update NewslettersController.php\n\nTest if an...,diff --git a/www/guis/users/application/contro...


In [4]:
train_code_change = list(train['diff'].astype(str))
test_code_change = list(test['diff'].astype(str))

In [5]:
# !pip install setfit
from commitfit import get_templated_dataset,sample_dataset
from datasets import Dataset, load_metric

In [6]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [7]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['positive','negative'], sample_size=8)

In [8]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['label', 'text', 'diff'],
    num_rows: 56
})

In [9]:
len(train)

40

In [10]:
train['label'].value_counts()

label
negative    25
positive    15
Name: count, dtype: int64

In [11]:
test['label'].value_counts()

label
negative    6322
positive    3750
Name: count, dtype: int64

In [12]:
len(train_dataset)

56

In [13]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    precision_score = metrics.precision_score(y_test,y_pred, average='weighted')
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"accuracy": accuracy_score, "precision": precision_score,"recall": recall_score, "f1":f1_score}

In [14]:
test['label'].value_counts()

label
negative    6322
positive    3750
Name: count, dtype: int64

In [15]:
train_dataset

Dataset({
    features: ['label', 'text', 'diff'],
    num_rows: 56
})

In [16]:
train

,label,text,diff
0,positive,Check types to avoid invalid reads/writes.,diff --git a/src/file.c b/src/file.c\nindex 4d...
1,positive,[CVE-2012-2098] Integrate fallback sort into t...,diff --git a/src/main/java/org/apache/commons/...
2,negative,Merge branch '1.8.x',diff --git a/src/ClientCommand.cpp b/src/Clien...
3,negative,[JBPM-9474] Produce events to Kafka by Signal ...,diff --git a/jbpm-bpmn2/src/main/java/org/jbpm...
4,positive,Resolving Issue #918\n\nMinor XSS and create g...,diff --git a/docs/CHANGELOG b/docs/CHANGELOG\n...
5,positive,CVE-2017-12895/ICMP: Check the availability of...,diff --git a/print-icmp.c b/print-icmp.c\ninde...
6,negative,9.0.4,diff --git a/package.json b/package.json\ninde...
7,negative,"Needs to allow this syscall as well, caught on...",diff --git a/linux_priv.c b/linux_priv.c\ninde...
8,positive,SPOOLSS: Try to avoid an infinite loop.\n\nUse...,diff --git a/epan/dissectors/packet-dcerpc-spo...
9,positive,Update NewslettersController.php\n\nTest if an...,diff --git a/www/guis/users/application/contro...


In [17]:
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"

In [18]:
from commitfit import CommitFitModel
model = CommitFitModel.from_pretrained(model_id)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [19]:
from commitfit import CommitFitTrainer

trainer = CommitFitTrainer(
    model=model,
    train_dataset=train_dataset,
    train_code_change = train_code_change,
    test_code_change = test_code_change,
    eval_dataset=test_dataset,
    metric = compute_metrics,
    num_iterations=20,
    num_epochs=1
)

In [21]:
%%time
trainer.train()
fewshot_metrics = trainer.evaluate()
fewshot_metrics

Generating Training Pairs: 100%|██████████| 20/20 [00:00<00:00, 228.46it/s]
***** Running training *****
  Num examples = 2240
  Num epochs = 1
  Total optimization steps = 140
  Total train batch size = 16
Epoch: 100%|██████████| 1/1 [00:29<00:00, 29.98s/it]
***** Running evaluation *****


CPU times: user 15min 20s, sys: 1min 16s, total: 16min 37s
Wall time: 4min 23s


/usr/local/miniconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'accuracy': 0.7594320889594917,
 'precision': 0.7554527929736741,
 'recall': 0.7594320889594917,
 'f1': 0.7554219369180497}